In [57]:
import garminconnect
import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from supabase import create_client
import os

load_dotenv()

True

In [43]:
email = os.getenv("email")
password = os.getenv("garminpassword")\

garmin = garminconnect.Garmin(email, password)
garmin.login()

garmin.display_name

'5f3dc0bc-4916-4755-894b-cca99162502b'

**Export sleep movment data to timeseries table** 

In [44]:
def safe_dataframe(data, key):
    """
    Safely extracts a nested value from a dictionary and returns a DataFrame.
    Handles missing keys, None values, scalars, or poorly structured dicts.
    """
    nested = data.get(key)

    if not nested:
        return pd.DataFrame()

    # If it's a dict but all values are scalars (not dicts/lists), skip it
    if isinstance(nested, dict):
        if all(not isinstance(v, (dict, list)) for v in nested.values()):
            return pd.DataFrame()
        return pd.DataFrame([nested])  
    
    # If list of struct objects or list
    if isinstance(nested, list):
        return pd.DataFrame(nested)

    # otherwise return empty 
    return pd.DataFrame()

In [45]:
import json
date = '2025-06-12'

def getdata(date):
    return garmin.get_sleep_data(date)
    

def getgeneral(data):
        return safe_dataframe(data, 'dailySleepDTO')

data = getdata(date)
df_general = getgeneral(data)

print(df_general.head())

              id  userProfilePK calendarDate  sleepTimeSeconds  \
0  1749687960000       80897268   2025-06-12             24660   

   napTimeSeconds  sleepWindowConfirmed sleepWindowConfirmationType  \
0               0                  True    enhanced_confirmed_final   

   sleepStartTimestampGMT  sleepEndTimestampGMT  sleepStartTimestampLocal  \
0           1749687960000         1749713640000             1749691560000   

   ...  lowestRespirationValue highestRespirationValue awakeCount  \
0  ...                    10.0                    20.0          1   

  avgSleepStress ageGroup  sleepScoreFeedback  sleepScoreInsight  \
0           19.0    ADULT       POSITIVE_DEEP               NONE   

   sleepScorePersonalizedInsight  \
0                  NOT_AVAILABLE   

                                         sleepScores  sleepVersion  
0  {'totalDuration': {'qualifierKey': 'FAIR', 'op...             2  

[1 rows x 38 columns]


In [46]:
def getsleepmovement(data):
    
    df = safe_dataframe(data, 'sleepMovement')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'])
    if 'endGMT' in df.columns:
        df['endGMT'] = pd.to_datetime(df['endGMT'])
    return df
   
df_sleep_movement = getsleepmovement(data)
print(df_sleep_movement.head())

             startGMT              endGMT  activityLevel
0 2025-06-11 23:26:00 2025-06-11 23:27:00       5.520759
1 2025-06-11 23:27:00 2025-06-11 23:28:00       5.036847
2 2025-06-11 23:28:00 2025-06-11 23:29:00       4.529466
3 2025-06-11 23:29:00 2025-06-11 23:30:00       4.008012
4 2025-06-11 23:30:00 2025-06-11 23:31:00       3.482673


**spo2 Data**

In [47]:
def getspo2(data):

    df = safe_dataframe(data, 'wellnessEpochSPO2DataDTOList')
    if 'epochTimestamp' in df.columns:
        df['epochTimestamp'] = pd.to_datetime(df['epochTimestamp'])
    return df
    
df_spo2 = getspo2(data)
print(df_spo2.head())

   userProfilePK      epochTimestamp    deviceId           calendarDate  \
0       80897268 2025-06-12 00:26:00  3339097678  2025-06-12T00:00:00.0   
1       80897268 2025-06-12 00:27:00  3339097678  2025-06-12T00:00:00.0   
2       80897268 2025-06-12 00:28:00  3339097678  2025-06-12T00:00:00.0   
3       80897268 2025-06-12 00:29:00  3339097678  2025-06-12T00:00:00.0   
4       80897268 2025-06-12 00:30:00  3339097678  2025-06-12T00:00:00.0   

   epochDuration  spo2Reading  readingConfidence  
0             60           95                 17  
1             60           96                 14  
2             60           96                 25  
3             60           94                 19  
4             60           94                  6  


In [48]:
def getrespiration(data):
    df = safe_dataframe(data, 'wellnessEpochRespirationDataDTOList')
    if 'startTimeGMT' in df.columns:
        df['startTimeGMT'] = pd.to_datetime(df['startTimeGMT'], unit='ms')
    
    return df
    
df_respiration = getrespiration(data)
print(df_respiration.head())

         startTimeGMT  respirationValue
0 2025-06-12 00:26:00              13.0
1 2025-06-12 00:26:00              13.0
2 2025-06-12 00:28:00              13.0
3 2025-06-12 00:30:00              13.0
4 2025-06-12 00:32:00              13.0


In [49]:
def getsleepHR(data):
    
    df = safe_dataframe(data, 'sleepHeartRate')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'], unit='ms')
    
    return df

df_sleephr = getsleepHR(data)
print(df_sleephr.head())

   value            startGMT
0     82 2025-06-12 00:26:00
1     82 2025-06-12 00:28:00
2     79 2025-06-12 00:30:00
3     76 2025-06-12 00:32:00
4     78 2025-06-12 00:34:00


In [50]:
def getsleepstress(data):
    df = safe_dataframe(data, 'sleepStress')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'], unit='ms')
    
    return df
    
df_sleep_stress = getsleepstress(data)
print(df_sleep_stress.head())

   value            startGMT
0     80 2025-06-12 00:24:00
1     70 2025-06-12 00:27:00
2     52 2025-06-12 00:30:00
3     46 2025-06-12 00:33:00
4     24 2025-06-12 00:36:00


In [51]:
def getbodybattery(data):
    df = safe_dataframe(data, 'sleepBodyBattery')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'], unit='ms')
    
    return df


df_sleep_BB = getbodybattery(data)
print(df_sleep_BB)

     value            startGMT
0       10 2025-06-12 00:24:00
1       10 2025-06-12 00:27:00
2       10 2025-06-12 00:30:00
3       10 2025-06-12 00:33:00
4       10 2025-06-12 00:36:00
..     ...                 ...
139     75 2025-06-12 07:21:00
140     75 2025-06-12 07:24:00
141     75 2025-06-12 07:27:00
142     75 2025-06-12 07:30:00
143     75 2025-06-12 07:33:00

[144 rows x 2 columns]


In [52]:
def getHRV(data):
    df = safe_dataframe(data, 'hrvData')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'], unit='ms')
    
    return df
   

df_sleep_HRV = getHRV(data)
print(df_sleep_HRV)

    value            startGMT
0    22.0 2025-06-12 00:26:14
1    23.0 2025-06-12 00:31:14
2    33.0 2025-06-12 00:36:14
3    42.0 2025-06-12 00:41:14
4    53.0 2025-06-12 00:46:14
..    ...                 ...
81   60.0 2025-06-12 07:11:14
82   57.0 2025-06-12 07:16:14
83   58.0 2025-06-12 07:21:14
84   57.0 2025-06-12 07:26:14
85   53.0 2025-06-12 07:31:14

[86 rows x 2 columns]


In [56]:
def getLevels(data):
    sleep_levels_data = data['sleepLevels']
    df = safe_dataframe(data, 'hrvData')
    if 'startGMT' in df.columns:
        df['startGMT'] = pd.to_datetime(df['startGMT'], unit='ms')
    if 'endGMT' in df.columns:
        df['endGMT'] = pd.to_datetime(df['endGMT'], unit='ms')
    return df
    

df_sleep_levels = getLevels(data)
print(df_sleep_levels)

    value            startGMT
0    22.0 2025-06-12 00:26:14
1    23.0 2025-06-12 00:31:14
2    33.0 2025-06-12 00:36:14
3    42.0 2025-06-12 00:41:14
4    53.0 2025-06-12 00:46:14
..    ...                 ...
81   60.0 2025-06-12 07:11:14
82   57.0 2025-06-12 07:16:14
83   58.0 2025-06-12 07:21:14
84   57.0 2025-06-12 07:26:14
85   53.0 2025-06-12 07:31:14

[86 rows x 2 columns]


In [54]:
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
dbname = os.getenv("dbname")

DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(DATABASE_URL)

try:
    with engine.connect() as conn:
        result = result = conn.execute(text("SELECT version();"))
        for row in result:
            print("Connected to:", row[0])
except Exception as e:
    print(" Connection failed:", e)

Connected to: PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


In [55]:
def processupload(date, engine):
    data = getdata(date)
    
    if not data or 'dailySleepDTO' not in data or not data['dailySleepDTO']:
        print(f"Missing data for {date}")
        return None  

    
    df_general = getgeneral(data)
    df_sleep_levels = getLevels(data)
    df_sleep_HRV = getHRV(data)
    df_sleep_BB = getbodybattery(data)
    df_sleep_stress = getsleepstress(data)
    df_sleephr = getsleepHR(data)
    df_respiration = getrespiration(data)
    df_spo2 = getspo2(data)

    frames = {
        "sleep_general": df_general,
        "sleep_levels": df_sleep_levels,
        "sleep_hrv": df_sleep_HRV,
        "sleep_BB": df_sleep_BB,
        "sleep_stress": df_sleep_stress,
        "sleep_hr": df_sleephr,
        "sleep_respiration": df_respiration,
        "sleep_spo2": df_spo2
    }

    for df in frames.values():
        for col in df.columns:
            # Convert dict columns to JSON string
            if df[col].apply(type).eq(dict).any():
                df[col] = df[col].apply(json.dumps)

    for table_name, df in frames.items():
        df['calendarDate'] = date
        df.to_sql(table_name, con=engine, if_exists="append", index=False)
        
    return True


def loopbackwards(start_date_str, engine, max_misses=7):
    current_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d").date()
    misses = 0

    while misses < max_misses:
        print(f"Processing date: {current_date}")
        success = processupload(str(current_date), engine)

        if success is None:
            print(f"No sleep data for {current_date}. Skipping.")
            misses += 1
        else:
            misses = 0  

        current_date -= datetime.timedelta(days=1)
    
    print("Stopped after too many missing days.")




today = datetime.date.today().strftime("%Y-%m-%d")
loopbackwards(today, engine)


Processing date: 2025-06-23
Processing date: 2025-06-22
Processing date: 2025-06-21
Processing date: 2025-06-20
Processing date: 2025-06-19
Processing date: 2025-06-18
Processing date: 2025-06-17
Processing date: 2025-06-16
Processing date: 2025-06-15
Processing date: 2025-06-14
Processing date: 2025-06-13
Processing date: 2025-06-12
Processing date: 2025-06-11
Processing date: 2025-06-10
Processing date: 2025-06-09
Processing date: 2025-06-08
Processing date: 2025-06-07
Processing date: 2025-06-06
Processing date: 2025-06-05
Processing date: 2025-06-04
Processing date: 2025-06-03
Processing date: 2025-06-02
Processing date: 2025-06-01
Processing date: 2025-05-31
Processing date: 2025-05-30
Processing date: 2025-05-29
Processing date: 2025-05-28
Processing date: 2025-05-27
Processing date: 2025-05-26
Processing date: 2025-05-25
Processing date: 2025-05-24
Processing date: 2025-05-23
Processing date: 2025-05-22
Processing date: 2025-05-21
Processing date: 2025-05-20
Processing date: 202

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

Creating a Whoop Like Recovery Score

In [ ]:
sleep_response = supabase.table("sleep_general").select("calendarDate, sleepTimeSeconds").execute()
sleep_df = pd.DataFrame(sleep_response.data)

